# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [38]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
# Create a Data Frame that loads the csv file exported in Part One
my_cities_df = pd.read_csv("Output/City_Data_Farenheit.csv", encoding="utf-8")

# Print the number of rows in the DataFrame (just to check that it matches the rows exported to the csv file in Part One)
print(len(my_cities_df))

# Display the first 5 rows of the DataFrame
my_cities_df.head()


579


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Kupang,20,ID,1584113216,100,-10.17,123.58,75.20,1.12
1,São Filipe,0,CV,1584112915,54,14.90,-24.50,77.63,15.23
2,Hilo,1,US,1584113198,82,19.73,-155.09,68.00,3.36
3,Letka,97,RU,1584113385,84,59.60,49.42,34.43,9.91
4,Aranda de Duero,84,ES,1584113387,57,41.67,-3.69,61.25,7.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Create a DataFrame called locations
locations = my_cities_df[["Lat", "Lng"]]

# Create a DataFrame called humidity
humidity = my_cities_df["Humidity"].astype(float)

# Create a variable called for the max humidity
max_humidity = humidity.max()

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_humidity, point_radius=4)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
# Create a Data Frame for cities with good weather based on our criteria
my_good_weather_cities_df = my_cities_df.loc[(my_cities_df["Max Temp"] > 50) 
                   & (my_cities_df["Max Temp"] < 80) 
                   & (my_cities_df["Cloudiness"] == 0) 
                   & (my_cities_df["Wind Speed"] < 10), :]

# Drop any rows that do not meet all of our criteria for good weather
my_good_weather_cities_df = my_good_weather_cities_df.dropna(how='any')

# Reset the index for the Data Frame
my_good_weather_cities_df.reset_index(inplace=True)

# Delete the old Index
del my_good_weather_cities_df['index']

# Check the number of rows in the DataFrame
print(len(my_good_weather_cities_df))

# Display the top 40 rows of the DataFrame
my_good_weather_cities_df.head(40)


38


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Tateyama,0,JP,1584113398,58,34.98,139.87,57.20,8.05
1,Lashio,0,MM,1584113487,39,22.93,97.75,69.35,2.37
2,Nelson Bay,0,AU,1584113067,88,-32.72,152.15,64.99,4.70
3,Le Locle,0,CH,1584113335,34,47.06,6.75,53.01,4.70
4,Düzce,0,TR,1584113534,77,40.84,31.16,52.07,4.03
5,Hermanus,0,ZA,1584113102,73,-34.42,19.23,75.99,5.01
6,San Quintín,0,MX,1584113235,79,30.48,-115.95,59.09,7.61
7,Dingle,0,PH,1584113190,76,11.00,122.67,74.39,8.21
8,Lagoa,0,PT,1584113526,67,39.05,-27.98,59.99,7.27
9,Valparaíso,0,CL,1584113093,72,-33.04,-71.63,66.20,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
# Add an empty column called hotel name if it does not exist 
if 'Hotel Name' not in my_good_weather_cities_df.columns:
    my_good_weather_cities_df.insert(loc=1, column='Hotel Name', value=['' for i in range(my_good_weather_cities_df.shape[0])])


# Iterate through the DataFrame and get the hotel name within the specified parameters for the cities with good weather
for index, row in my_good_weather_cities_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    params = {
        "location": f"{lat},{lng}",
        "rankby": "distance",  
        "keyword":"hotel",
        "key": g_key,
        
    }
   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response_json = response.json()
    try:
        my_good_weather_cities_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
    except(KeyError, IndexError):
        print("Missing field/result... skipping.")
              
my_good_weather_cities_df


Missing field/result... skipping.


,City,Hotel Name,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Tateyama,tu.ne.Hostel,0,JP,1584113398,58,34.98,139.87,57.20,8.05
1,Lashio,Gold Mount Hotel,0,MM,1584113487,39,22.93,97.75,69.35,2.37
2,Nelson Bay,Port Stephens Motel,0,AU,1584113067,88,-32.72,152.15,64.99,4.70
3,Le Locle,Hôtel La Fleur de Lis,0,CH,1584113335,34,47.06,6.75,53.01,4.70
4,Düzce,DÜZCE HAN OTEL,0,TR,1584113534,77,40.84,31.16,52.07,4.03
5,Hermanus,The Gables,0,ZA,1584113102,73,-34.42,19.23,75.99,5.01
6,San Quintín,Hotel Jardines Baja,0,MX,1584113235,79,30.48,-115.95,59.09,7.61
7,Dingle,Oyo 236 Hotel Edmundo,0,PH,1584113190,76,11.00,122.67,74.39,8.21
8,Lagoa,Casa da Madrinha Graciosa,0,PT,1584113526,67,39.05,-27.98,59.99,7.27
9,Valparaíso,Hotel Casa Higueras,0,CL,1584113093,72,-33.04,-71.63,66.20,5.82


In [43]:
# Drop the rows which do not match all the criteria
my_good_weather_cities_df = my_good_weather_cities_df.dropna(how='any')

# Dispaly the top 40 rows of the DataFrame
my_good_weather_cities_df.head(40)


,City,Hotel Name,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Tateyama,tu.ne.Hostel,0,JP,1584113398,58,34.98,139.87,57.20,8.05
1,Lashio,Gold Mount Hotel,0,MM,1584113487,39,22.93,97.75,69.35,2.37
2,Nelson Bay,Port Stephens Motel,0,AU,1584113067,88,-32.72,152.15,64.99,4.70
3,Le Locle,Hôtel La Fleur de Lis,0,CH,1584113335,34,47.06,6.75,53.01,4.70
4,Düzce,DÜZCE HAN OTEL,0,TR,1584113534,77,40.84,31.16,52.07,4.03
5,Hermanus,The Gables,0,ZA,1584113102,73,-34.42,19.23,75.99,5.01
6,San Quintín,Hotel Jardines Baja,0,MX,1584113235,79,30.48,-115.95,59.09,7.61
7,Dingle,Oyo 236 Hotel Edmundo,0,PH,1584113190,76,11.00,122.67,74.39,8.21
8,Lagoa,Casa da Madrinha Graciosa,0,PT,1584113526,67,39.05,-27.98,59.99,7.27
9,Valparaíso,Hotel Casa Higueras,0,CL,1584113093,72,-33.04,-71.63,66.20,5.82


In [44]:
my_good_weather_cities_df = my_good_weather_cities_df.reset_index()


In [45]:
my_good_weather_cities_df.head(40)


,index,City,Hotel Name,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Tateyama,tu.ne.Hostel,0,JP,1584113398,58,34.98,139.87,57.20,8.05
1,1,Lashio,Gold Mount Hotel,0,MM,1584113487,39,22.93,97.75,69.35,2.37
2,2,Nelson Bay,Port Stephens Motel,0,AU,1584113067,88,-32.72,152.15,64.99,4.70
3,3,Le Locle,Hôtel La Fleur de Lis,0,CH,1584113335,34,47.06,6.75,53.01,4.70
4,4,Düzce,DÜZCE HAN OTEL,0,TR,1584113534,77,40.84,31.16,52.07,4.03
5,5,Hermanus,The Gables,0,ZA,1584113102,73,-34.42,19.23,75.99,5.01
6,6,San Quintín,Hotel Jardines Baja,0,MX,1584113235,79,30.48,-115.95,59.09,7.61
7,7,Dingle,Oyo 236 Hotel Edmundo,0,PH,1584113190,76,11.00,122.67,74.39,8.21
8,8,Lagoa,Casa da Madrinha Graciosa,0,PT,1584113526,67,39.05,-27.98,59.99,7.27
9,9,Valparaíso,Hotel Casa Higueras,0,CL,1584113093,72,-33.04,-71.63,66.20,5.82


In [46]:
del my_good_weather_cities_df['index']


In [47]:
my_good_weather_cities_df.head(40)


,City,Hotel Name,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Tateyama,tu.ne.Hostel,0,JP,1584113398,58,34.98,139.87,57.20,8.05
1,Lashio,Gold Mount Hotel,0,MM,1584113487,39,22.93,97.75,69.35,2.37
2,Nelson Bay,Port Stephens Motel,0,AU,1584113067,88,-32.72,152.15,64.99,4.70
3,Le Locle,Hôtel La Fleur de Lis,0,CH,1584113335,34,47.06,6.75,53.01,4.70
4,Düzce,DÜZCE HAN OTEL,0,TR,1584113534,77,40.84,31.16,52.07,4.03
5,Hermanus,The Gables,0,ZA,1584113102,73,-34.42,19.23,75.99,5.01
6,San Quintín,Hotel Jardines Baja,0,MX,1584113235,79,30.48,-115.95,59.09,7.61
7,Dingle,Oyo 236 Hotel Edmundo,0,PH,1584113190,76,11.00,122.67,74.39,8.21
8,Lagoa,Casa da Madrinha Graciosa,0,PT,1584113526,67,39.05,-27.98,59.99,7.27
9,Valparaíso,Hotel Casa Higueras,0,CL,1584113093,72,-33.04,-71.63,66.20,5.82


In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in my_good_weather_cities_df.iterrows()]
locations = my_good_weather_cities_df[["Lat", "Lng"]]


In [49]:
print(hotel_info)
print(locations)


['\n<dl>\n<dt>Name</dt><dd>tu.ne.Hostel</dd>\n<dt>City</dt><dd>Tateyama</dd>\n<dt>Country</dt><dd>JP</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Gold Mount Hotel</dd>\n<dt>City</dt><dd>Lashio</dd>\n<dt>Country</dt><dd>MM</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Port Stephens Motel</dd>\n<dt>City</dt><dd>Nelson Bay</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Hôtel La Fleur de Lis</dd>\n<dt>City</dt><dd>Le Locle</dd>\n<dt>Country</dt><dd>CH</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>DÜZCE HAN OTEL</dd>\n<dt>City</dt><dd>Düzce</dd>\n<dt>Country</dt><dd>TR</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>The Gables</dd>\n<dt>City</dt><dd>Hermanus</dd>\n<dt>Country</dt><dd>ZA</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Hotel Jardines Baja</dd>\n<dt>City</dt><dd>San Quintín</dd>\n<dt>Country</dt><dd>MX</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Oyo 236 Hotel Edmundo</dd>\n<dt>City</dt><dd>Dingle</dd>\n<dt>Country</dt><dd>PH</dd>\n</dl>\n', '\n<dl>\n<dt>Name</dt><dd>Casa da Mad

In [50]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info, hover_text=my_good_weather_cities_df["Hotel Name"])

# Add the layer to the map
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))